<a href="https://colab.research.google.com/github/anand-therattil/wine_quality/blob/main/Wine_quality_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import svm
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

In [2]:
# To be used for the drive connection
drive.mount(r'/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv(r'drive/MyDrive/Colab Notebooks/Dataset/winequality.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
class pre_processing:
  def __init__(self,data):
    self.data = data
  
  def nan_replace(self):
    self.data = self.data.apply(lambda x: x.fillna(x.value_counts().index[0]))
    return  self.data

  def standard(self,independent):
    standardized_data = StandardScaler().fit_transform(independent)
    return standardized_data
    
  def cat2num(self,column_name,LE_OHE):
    if(LE_OHE=="LE"):
      label_encoder = preprocessing.LabelEncoder()
      column_data =  self.data[column_name].apply(label_encoder.fit_transform)
      self.data.drop(column_name,axis=1,inplace=True)
      self.data = pd.concat([self.data,column_data],axis=1)
    elif(LE_OHE=="OHE"):
      Onehotencoder = OneHotEncoder()
      column_data = Onehotencoder.fit_transform(self.data[column_name]).toarray()
      column_data = pd.DataFrame(column_data)
      self.data.drop(column_name,axis=1,inplace=True)
      self.data = pd.concat([self.data,column_data],axis=1)
      self.data.columns = self.data.columns.astype(str)
    return  self.data

In [14]:
class model:
  def __init__(self,independent_data,dependent_data):
    self.independent_data = independent_data
    self.dependent_data = dependent_data
    self.model = None
    self.predict_value = None

  def Linear_Regression(self):
    self.model = LinearRegression()
    self.model.fit(self.independent_data,self.dependent_data)
    return self.model

  def XGB_Regressor(self):
    self.model = XGBRegressor()
    self.model.fit(self.independent_data,self.dependent_data)
    return self.model

  def Logistic_Regression(self):
    self.model = LogisticRegression()
    self.model.fit(self.independent_data,self.dependent_data)
    return self.model

  def SVM(self):
    self.model = svm.SVC(kernel='rbf')
    self.model.fit(self.independent_data,self.dependent_data)
    return self.model 

  def predict(self,model,test_data):
    self.model = model
    self.predict_value = self.model.predict(test_data)
    return self.predict_value

In [15]:
independent = data.drop('quality',axis=1)
dependent = data['quality']

pre_processing_model = pre_processing(data)
independent = pre_processing_model.standard(independent)
independent = pd.DataFrame(independent)


In [16]:
train_ind,test_ind,train_dep,test_dep = train_test_split(independent,dependent,test_size=0.2,random_state=1)
model_class = model(train_ind,train_dep)
model1 = model_class.SVM()

In [17]:
predict_values = model_class.predict(model1,train_ind)

In [18]:
test_acc = accuracy_score(predict_values,train_dep)

In [19]:
test_acc

0.674745895230649